In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
!pip install datasets
!pip install -U bitsandbytes
!pip install peft

In [ ]:
import torch
import io
import pickle
import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) غ
Invalid input. Must be one of ('y', 'yes', '1', 'n', 'no', '0', '')
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `1` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate

In [ ]:
import torch
from peft import LoraConfig
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics2ForConditionalGeneration

DEVICE = "cuda:0"
USE_LORA = False
USE_QLORA = True

processor = AutoProcessor.from_pretrained(
    "HuggingFaceM4/idefics2-8b",
    do_image_splitting=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Chat templates should be in a 'chat_template.jinja' file but found key='chat_template' in the processor's config. Make sure to move your template to its own file.


preprocessor_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics2ForConditionalGeneration
from peft import LoraConfig

if USE_QLORA or USE_LORA:
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules='.*(text_model|modality_projection|perceiver_resampler).*(down_proj|gate_proj|up_proj|k_proj|q_proj|v_proj|o_proj).*$',
        use_dora=False if USE_QLORA else True,
        init_lora_weights="gaussian"
    )

    # If using QLoRA, configure Bits and Bytes
    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
    else:
        bnb_config = None

    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
    )

    model.add_adapter(lora_config)
    model.enable_adapters()
else:
    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        _attn_implementation="flash_attention_2",
    ).to(DEVICE)



config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/74.4k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.64G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
dataset = load_dataset("mdwiratathya/SLAKE-vqa-english")


README.md:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/31.1M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.34M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/9.59M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4919 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1053 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1061 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
from torch.utils.data import Dataset
from PIL import Image
import random


class VQADataset(Dataset):
    def __init__(self, hf_dataset, processor):
        self.dataset = hf_dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        image = self.dataset[idx]['image']  # This is a PIL image

        return {
            "image": image,  # Directly use the PIL image
            "query": {"en": question},
            "answers": [answer]
        }

# Define the MyDataCollator class
class MyDataCollator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, examples):
        texts = []
        images = []
        for example in examples:
            image = example["image"]  # Directly use the PIL image
            question = example["query"]['en']
            answer = random.choice(example["answers"])

            # Create messages as before
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Answer briefly."},
                        {"type": "image"},
                        {"type": "text", "text": question}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": answer}
                    ]
                }
            ]
            text = self.processor.apply_chat_template(messages, add_generation_prompt=False)
            texts.append(text.strip())
            images.append(image)

        # Process texts and images using the processor
        batch = self.processor(text=texts, images=images, return_tensors="pt", padding=True)

        # Create labels for training
        labels = batch["input_ids"].clone()
        labels[labels == self.processor.tokenizer.pad_token_id] = -100  # Ignore padding in loss calculation
        batch["labels"] = labels

        return batch


train_dataset = VQADataset(hf_dataset=dataset['train'], processor=processor)
test_dataset = VQADataset(hf_dataset=dataset['test'], processor=processor)
valid_dataset = VQADataset(hf_dataset=dataset['validation'], processor=processor)

data_collator = MyDataCollator(processor=processor)


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback


training_args = TrainingArguments(
    output_dir="IDEFICS2",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=50,  # Increased evaluation frequency
    save_steps=50,
    max_steps=100,
    logging_steps=5,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["labels"],
    report_to="none",
    optim="paged_adamw_8bit",
    weight_decay=0.05,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()


Step,Training Loss,Validation Loss
50,0.138500,0.140658
100,0.118400,0.106247


TrainOutput(global_step=100, training_loss=0.4403654184937477, metrics={'train_runtime': 3304.9525, 'train_samples_per_second': 0.242, 'train_steps_per_second': 0.03, 'total_flos': 3635666079123936.0, 'train_loss': 0.4403654184937477, 'epoch': 0.16263468184590363})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,LlavaProcessor

# Define save path
save_directory = "/content/drive/MyDrive/VQA12Feb/models/IdeficsFinalSlake"


from transformers import AutoConfig

# config = AutoConfig.from_pretrained("HuggingFaceM4/Idefics3-8B-Llama3")  # Use the base model
model.config.save_pretrained(save_directory)
model.save_pretrained(save_directory)
# processor = LlavaProcessor.from_pretrained("HuggingFaceM4/Idefics3-8B-Llama3")
processor.save_pretrained(save_directory)

print(f"✅ Model saved at: {save_directory}")

print(f"✅ Model saved at: {save_directory}")


✅ Model saved at: /content/drive/MyDrive/VQA12Feb/models/IdeficsFinalSlake
✅ Model saved at: /content/drive/MyDrive/VQA12Feb/models/IdeficsFinalSlake


In [ ]:
from accelerate.utils import release_memory
# ...

release_memory(model)

[None]

In [ ]:
test_results = trainer.evaluate(test_dataset)

# Print the evaluation results
print(test_results)

{'eval_loss': 0.10028829425573349, 'eval_runtime': 1147.0363, 'eval_samples_per_second': 0.925, 'eval_steps_per_second': 0.925, 'epoch': 0.16263468184590363}


In [ ]:
t_results = trainer.evaluate()

print(t_results)

{'eval_loss': 0.10624716430902481, 'eval_runtime': 1104.4676, 'eval_samples_per_second': 0.953, 'eval_steps_per_second': 0.953, 'epoch': 0.16263468184590363}


In [ ]:
from huggingface_hub import HfApi

# Set up the model and tokenizer save path
save_directory = "/content/drive/MyDrive/VQA12Feb/models/IdeficsFinalSlake"
repo_name = "IdeficsFinal20FSlakelo"  # Specify the name of your model repository
user_name = "Dtarget"  # Your Hugging Face username

# Initialize Hugging Face API
api = HfApi()

# Create the repository as private
api.create_repo(repo_id=f"{user_name}/{repo_name}", private=True, repo_type="model")

# Upload model to Hugging Face
api.upload_folder(
    folder_path=save_directory,
    repo_id=f"{user_name}/{repo_name}",
    path_in_repo=""  # Upload to root of the repository
)

print(f"✅ Model uploaded to Hugging Face privately: https://huggingface.co/{user_name}/{repo_name}")


adapter_model.safetensors:   0%|          | 0.00/93.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model uploaded to Hugging Face privately: https://huggingface.co/Dtarget/IdeficsFinal20FSlakelo
